In [0]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Load in the pre-pitcher pre-processed df

In [0]:
#load in pickled pre_processed df

df1 = pd.read_pickle("pre_pitcher_pre_processed_df_part1.pkl", compression='zip')
df2 = pd.read_pickle("pre_pitcher_pre_processed_df_part2.pkl", compression='zip')

df = pd.concat([df1, df2])

In [134]:
#clear some memory:
del df1
del df2
gc.collect()

28

## Choose the pitcher to model

In [135]:
df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
pitcher,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
434378.0,6188,6188,6188,6079,6076,6076,6188,6076,1369,6188,6076,6076,6076,6076,6188,6188,6188,6188,6188,894,894,6130,6076,6076,6076,6076,6076,6076,6130,6130,1568,1669,1669,6077,6070,6078,6188,6076,901,901,1513,1513,1513,1513,901,6188,6188,6188,6188,6188,6188,6188,6188,6188,6188,6188,6188,6188,6188,6188,901,6188,6130,6130,6188,1325,6188,6188,6188,6188,6188,6188,6188,6188,5943,5943,5943,5943,5943,5943,5943,5743,5743,5743,5743,5743,5743,5743,5207,5207,5207,5207,5207,5207,5207,6076,6188,6188,6188,1513,6188,1513,6125,6130,6188,6188,6188
545333.0,6117,6117,6117,6114,6114,6114,6117,6114,1329,6117,6114,6114,6114,6114,6117,6117,6117,6117,6117,918,918,6116,6114,6114,6114,6114,6114,6114,6116,6116,1479,1548,1548,6114,5900,6114,6117,6114,921,921,1526,1526,1526,1526,921,6117,6117,6117,6117,6117,6117,6117,6117,6117,6117,6117,6117,6117,6117,6117,921,6117,6116,6116,6117,1725,6117,6117,6117,6117,6117,6117,6117,6117,5789,5789,5789,5789,5789,5789,5789,5478,5478,5478,5478,5478,5478,5478,4805,4805,4805,4805,4805,4805,4805,6006,6117,6117,6117,1526,6117,1526,6114,6116,6117,6117,6117
605400.0,6072,6072,6072,6071,6071,6071,6072,6071,1383,6072,6071,6071,6071,6071,6072,6072,6072,6072,6072,999,999,6072,6071,6071,6071,6071,6071,6071,6072,6072,1500,1593,1593,6071,6065,6071,6072,6071,1001,1001,1557,1557,1557,1557,1001,6072,6072,6072,6072,6072,6072,6072,6072,6072,6072,6072,6072,6072,6072,6072,1001,6072,6072,6070,6072,1556,6072,6072,6072,6072,6072,6072,6072,6072,5608,5608,5608,5608,5608,5608,5608,5298,5298,5298,5298,5298,5298,5298,4793,4793,4793,4793,4793,4793,4793,5965,6072,6072,6072,1557,6072,1557,6072,6070,6072,6072,6072
458681.0,5961,5961,5961,5839,5838,5838,5961,5838,1277,5961,5838,5838,5838,5838,5961,5961,5961,5961,5961,942,942,5959,5838,5838,5838,5838,5838,5838,5959,5959,1559,1644,1644,5830,5824,5839,5961,5838,944,944,1447,1447,1447,1447,944,5961,5961,5961,5961,5961,5961,5961,5961,5961,5961,5961,5961,5961,5961,5961,944,5961,5959,5951,5961,1948,5961,5961,5961,5961,5961,5961,5961,5961,5643,5643,5643,5643,5643,5643,5643,5328,5328,5328,5328,5328,5328,5328,4900,4900,4900,4900,4900,4900,4900,5778,5961,5961,5961,1447,5961,1447,5955,5951,5961,5961,5961
543037.0,5925,5925,5925,5923,5922,5922,5925,5922,1289,5925,5922,5922,5922,5922,5925,5925,5925,5925,5925,807,807,5925,5922,5922,5922,5922,5922,5922,5925,5925,1468,1540,1540,5923,5889,5923,5925,5922,811,811,1447,1447,1447,1447,811,5925,5925,5925,5925,5925,5925,5925,5925,5925,5925,5925,5925,5925,5925,5925,811,5925,5925,5924,5925,1480,5925,5925,5925,5925,5925,5925,5925,5925,558

In [136]:
#make pitcher_df subset where pitcher == picther_id

pitcher_id = 434378.0

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,1,1,Justin Verlander,NaN,ball,NaN,Slider,Standard,11,ahead,breaking
1513,FF,2018-03-29,22691,95.300003,-1.9823,6.7639,592261.0,434378.0,5.0,6.0,2018.0,-1.0804,1.4159,0.1783,2.3724,0.0,0.0,0.0,0.0,1.0,108.040001,149.179993,435263.0,7.9219,-138.185593,-8.8461,-15.272600,30.025200,-12.733900,3.1947,1.2860,145.0,104.199997,7.0,94.040001,2579.0,5.690,529411.0,54.808498,0.674,0.644,0.0,1.0,0.0,0.0,4.0,6.0,4.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,X,line_drive,Bot,180329_195226,Standard,1.0,NaN,False,False,False,False,0,1,1,0,66.241684,21.138212,42.050209,

In [0]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

pitcher_list = df['pitcher'].unique().tolist()

#on rare occassions in extra-inning games, a position player is called in to pitch: to avoid categorizing
#him as a pitcher we'll only count pitchers with over 100 pitches:
for pitcher in pitcher_list:
    if len(df[df['pitcher'] == pitcher]) < 100:
        pitcher_list.remove(pitcher)

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [0]:
#Todo: consider feature engineering more features here (see blueprint notebook ideas)
#Add code to update the pitcher tendencies iteratively month by month to avoid leakage (see pre-processing step2)

In [0]:
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id
df17 = pd.read_pickle('preprocessed_2017.pkl', compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

In [0]:
#call the pitcher scouting report function and add to the df:

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_type'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_type'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_tendency_features(pitcher_df, pitcher_tendencies_overall, pitcher_tendencies_by_count):
    df = pitcher_df.copy()
    
    pitch_types = pitcher_tendencies_overall.keys()
    
    for pitch_type in pitch_types:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
        
        def get_overall_perc(x):
            return pitcher_tendencies_overall[x]
        def get_by_count_perc(x):
            return pitcher_tendencies_by_count[x][pitch_type]
        
        df[overall_feature] = pitch_type
        df[overall_feature] = df[overall_feature].apply(get_overall_perc)
        df[count_cat_feature] = df['count_cat'].apply(get_by_count_perc)
    return df

In [0]:
start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

  
def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get the pitch tendencies from prior:
        pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, pitcher_tendencies_overall, pitcher_tendencies_by_count)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df

In [142]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

CPU times: user 739 ms, sys: 4.03 ms, total: 743 ms
Wall time: 746 ms


In [143]:
pitcher_df.isna().sum()

pitch_type                            0
game_date                             0
index                                 0
release_speed                       109
release_pos_x                       112
release_pos_z                       112
batter                                0
pitcher                               0
zone                                112
hit_location                       4819
game_year                             0
pfx_x                               112
pfx_z                               112
plate_x                             112
plate_z                             112
on_3b                                 0
on_2b                                 0
on_1b                                 0
outs_when_up                          0
inning                                0
hc_x                               5294
hc_y                               5294
fielder_2                            58
vx0                                 112
vy0                                 112


## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [0]:
#Todo: add trailing 15 pitch features

In [0]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 'PH' (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 'PH'
            
    game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: True if x in pitcher_list else False)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == False, other='pitcher')
    return game_df
    
    
def make_game_pitchcount_and_trailing_pitch_features(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    #take the first game and make the pitch count feature
    first_game_df = df[df['game_pk'] == games[0]].copy()
    first_game_df['pitch_count'] = range(1, first_game_df.shape[0] + 1)
    
    #make the L1_pitch type feature:
    first_game_df['L1_pitch_type'] = first_game_df['pitch_type'].shift(periods=1)
    first_game_df['L1_pitch_result'] = first_game_df['type'].shift(periods=1)
    first_game_df['L1_pitch_result'] = first_game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
    first_game_df['L1_pitch_zone'] = first_game_df['zone'].shift(periods=1)
    first_game_df['L1_pitch_zone'] = first_game_df['L1_pitch_zone'].fillna(-1)
    
    #overall strike % (to fill in for first 5 pitches L5_strike_perc)
    overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100
    
    #make the trailing 5 pitches:
    for index, row in first_game_df.iterrows():
        #fill NaNs for L1_pitch using same method as when pitch_type was missing
        if row['pitch_count'] == 1:
            random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                               weights=list(pitcher_tendencies_overall.values()), 
                               k=1)[0]
            first_game_df.at[index, 'L1_pitch_type'] = random_pitch
            
        #for the first 5 rows, use overall pitcher tendencies    
        if row['pitch_count'] < 6:
            #fill with overall tendencies
            for pitch in list(pitcher_tendencies_overall.keys()):
                feature = 'L5_' + pitch + '_perc'
                first_game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100  
                #strike %
                first_game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
        else:
            current_pitch = first_game_df.at[index, 'pitch_count']
            #make a subset of the prev 5 pitches
            subset = first_game_df[(first_game_df['pitch_count'] > current_pitch - 6) & (first_game_df['pitch_count'] < current_pitch)]
            #grab the value count percentages for the last 5 pitches
            subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
            try:
                L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
            except KeyError:
                L5_strike_perc = 0
                
            first_game_df.at[index, 'L5_strike_perc'] = L5_strike_perc
            
            #iterate over all possible pitch types this pitcher throws:
            for pitch in list(pitcher_tendencies_overall.keys()):
                feature = 'L5_' + pitch + '_perc'
                #if he has thrown that pitch type in last 5
                try:
                    first_game_df.at[index, feature] = subset_percentages[pitch] * 100
                #except for when he hasnt thrown that type in last 5
                except:
                    first_game_df.at[index, feature] = 0
                    
    #apply the battting order features to the game:                
    first_game_df = make_game_batting_order(first_game_df)
    
    #iterate the same process for the rest of his games:
    for game in games[1:]:
        game_df = df[df['game_pk'] == game].copy() #get df for that game only
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1) #make the pitch count for the game
        game_df['L1_pitch_type'] = game_df['pitch_type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_pitch_zone'] = game_df['L1_pitch_zone'].fillna(0)
        
        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #fill NaNs for L1_pitch using same method as when pitch_type was missing
            if row['pitch_count'] == 1:
                random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                               weights=list(pitcher_tendencies_overall.values()), 
                               k=1)[0]
                game_df.at[index, 'L1_pitch_type'] = random_pitch
            
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    #strike %
                    game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
            else:
                current_pitch = game_df.at[index, 'pitch_count']
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
                
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0
                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    except:
                        game_df.at[index, feature] = 0
                        
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
        
        #concatenate that game w/ updated pitch count and trailing pitches w/ prev games
        if game_df['game_pk'].values[0] == games[1]:
            new_df = pd.concat([first_game_df, game_df]) #concat the game_df w/ the first game
        else:
            new_df = pd.concat([new_df, game_df]) #concat the game_df w/ the previous games
    return new_df

In [146]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features(pitcher_df, pitcher_list)

CPU times: user 36.4 s, sys: 21 ms, total: 36.4 s
Wall time: 36.5 s


In [147]:
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_CU_perc,count_cat_CU_perc,overall_CH_perc,count_cat_CH_perc,overall_FC_perc,count_cat_FC_perc,overall_FT_perc,count_cat_FT_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_FF_perc,L5_strike_perc,L5_SL_perc,L5_CU_perc,L5_CH_perc,L5_FC_perc,batting_order_slot,pitcher_AB
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,0.583272,0.685385,0.215498,0.163427,0.151046,0.114471,0.039606,0.029518,0.007626,0.005040,0.002952,0.002160,1,CU,first pitch,-1.0,56.076277,54.250162,24.676794,16.305753,2.60181,0.339367,1,False
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball,0.583272,0.493105,0.215498,0.257727,0.151046,0.198288,0.039606,0.039467,0.007626,0.008084,0.002952,0.003329,2,FF,S,8.0,56.076277,54.250162,24.676794,16.305753,2.60181,0.339367,1,False
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,4

## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [0]:
batting_order_slot_map = pickle.load(open("batting_order_slot_map.pkl", "rb"))

In [157]:
batting_order_slot_map

{1: {'breaking_babip': 0.1872640385570563,
  'breaking_bip_swung_perc': 36.89605241827341,
  'breaking_chase_perc': 31.577836069125617,
  'breaking_est_woba': 0.3397664965875092,
  'breaking_iso_value': 0.13101442385072887,
  'breaking_perc_faced': 26.340695895944986,
  'breaking_taken_strike_perc': 35.5666746896674,
  'fastball_babip': 0.2154025865714933,
  'fastball_bip_swung_perc': 40.45598656570413,
  'fastball_chase_perc': 27.84080495082605,
  'fastball_est_woba': 0.37223512074559534,
  'fastball_iso_value': 0.15691701622442916,
  'fastball_perc_faced': 62.765084187795196,
  'fastball_taken_strike_perc': 31.962136660953938,
  'offspeed_babip': 0.20294449910457169,
  'offspeed_bip_swung_perc': 40.37310697509579,
  'offspeed_chase_perc': 36.54483903027876,
  'offspeed_est_woba': 0.32781539532817017,
  'offspeed_iso_value': 0.13571896819088838,
  'offspeed_perc_faced': 10.883704814573973,
  'offspeed_taken_strike_perc': 23.32780224123913,
  'pitchout_perc_faced': 0.010515101697300902

In [0]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    return df

In [153]:
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: 9
finished w/ slot: pitcher
finished w/ slot: PH


In [154]:
pitcher_df.isna().sum()

pitch_type                            0
game_date                             0
index                                 0
release_speed                       109
release_pos_x                       112
release_pos_z                       112
batter                                0
pitcher                               0
zone                                112
hit_location                       4819
game_year                             0
pfx_x                               112
pfx_z                               112
plate_x                             112
plate_z                             112
on_3b                                 0
on_2b                                 0
on_1b                                 0
outs_when_up                          0
inning                                0
hc_x                               5294
hc_y                               5294
fielder_2                            58
vx0                                 112
vy0                                 112


In [161]:
pitcher_df.sort_values(by=['game_date', 'game_pk', 'at_bat_number']).head(10)

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_CU_perc,count_cat_CU_perc,overall_CH_perc,count_cat_CH_perc,overall_FC_perc,count_cat_FC_perc,overall_FT_perc,count_cat_FT_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_FF_perc,L5_strike_perc,L5_SL_perc,L5_CU_perc,L5_CH_perc,L5_FC_perc,batting_order_slot,pitcher_AB
1510,FF,2018-03-29,22730,94.599998,-1.9486,6.8034,592261.0,434378.0,8.0,NaN,2018.0,-0.8339,1.6529,0.2748,1.7070,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,7.4453,-136.895203,-10.8637,-11.885600,32.316799,-9.809500,3.3339,1.4562,NaN,NaN,NaN,92.489998,2652.0,5.498,529411.0,55.000801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,S,NaN,Bot,180329_195119,Standard,1.0,NaN,False,False,False,False,0,0,1,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,0,Justin Verlander,NaN,called_strike,NaN,4-Seam Fastball,Standard,00,neutral,fastball,0.583272,0.685385,0.215498,0.163427,0.151046,0.114471,0.039606,0.029518,0.007626,0.005040,0.002952,0.002160,1,CU,first pitch,-1.0,56.076277,54.250162,24.676794,16.305753,2.60181,0.339367,1,False
1511,FF,2018-03-29,22711,94.300003,-1.9916,6.6522,592261.0,434378.0,14.0,NaN,2018.0,-1.2892,1.4338,0.4047,1.2020,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.9916,-136.566605,-11.4425,-18.015499,31.964199,-12.041500,3.3446,1.4668,NaN,NaN,NaN,93.283997,2638.0,6.095,529411.0,54.403400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195137,Standard,1.0,NaN,False,True,False,False,0,0,0,0,66.241684,21.138212,42.050209,40.000000,0.300159,0.220395,0.098684,26.995565,27.516779,36.702129,43.915344,0.231478,0.200000,0.054545,6.762750,36.231884,21.666666,33.962265,0.426231,0.250000,0.041667,0.000000,0,1,Justin Verlander,NaN,ball,NaN,4-Seam Fastball,Standard,01,ahead,fastball,0.583272,0.493105,0.215498,0.257727,0.151046,0.198288,0.039606,0.039467,0.007626,0.008084,0.002952,0.003329,2,FF,S,8.0,56.076277,54.250162,24.676794,16.305753,2.60181,0.339367,1,False
1512,SL,2018-03-29,22699,86.400002,-2.1580,6.5734,592261.0,434378.0,14.0,NaN,2018.0,0.2707,0.3526,1.5138,-0.0429,0.0,0.0,0.0,0.0,1.0,NaN,NaN,435263.0,8.0583,-125.145203,-10.0136,1.186800,24.599800,-26.845699,3.3339,1.4562,NaN,NaN,NaN,84.944000,2680.0,5.543,529411.0,54.956001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,R,R,TEX,HOU,B,NaN,Bot,180329_195201,Standard,1.0,NaN,False,True,False,True,0,0,0,0,66.241684,21.138212,42.050209,4

## Add Pitcher / Batter Matchup History Report

In [0]:
##Todo

## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and June/July/August 2019 as Test)

In [0]:
#Todo